In [ ]:
import numpy as np 
import os 
import re
import plotly.graph_objects as go
import plotly.offline as pyo

In [ ]:
path = os.getcwd()
path

In [ ]:
#fig template 
fig_template = go.layout.Template()
fig_template.layout = {
    'template': 'simple_white+presentation',
    'autosize': False,
    'width': 800,
    'height': 600,
    # 'opacity': 0.2,
    'xaxis': {
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 1.5+0.5,
        'tickwidth': 1.5+0.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white',
        },
    'yaxis': {
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 1.5+0.5,
        'tickwidth': 1.5+0.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white'
        },
    'font':{'family':'mathjax',
            'size': 22,
            }
}

In [ ]:
def extract_number(file_name):
    match = re.search(r'\d+', file_name)
    return int(match.group()) if match else None 

In [ ]:
folder_path = path+'/data'

files=[]
for file in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file)
    if os.path.isfile(file_path):
      files.append(file)  

files_with_numbers = [(file_name, extract_number(file_name)) for file_name in files]
sorted_files_with_numbers = sorted(files_with_numbers, key=lambda x: (x[1] is None, x[1]))

sorted_files_with_numbers

In [ ]:
flake1=np.array([10,3,0.5])
flake2=np.array([10,3,1])
flake4=np.array([10,3,3])
flake3=np.array([10,4,0.5])
flake5=np.array([10,4,1])
flake6=np.array([10,4,3])
flake12=np.array([20,4,0.5])
flake13=np.array([20,3.5,3])
flake14=np.array([10,5,0.5])
flake15=np.array([10,5,1])
flake16=np.array([10,5,3])
flake17=np.array([20,5,3])
flake18=np.array([20,5,1])
flake19=np.array([20,5,0.5])
flake20=np.array([20,4,3])
flake21=np.array([20,4,1])
flake25=np.array([20,5,3])
flake26=np.array([20,5,1])
flake27=np.array([20,5,0.5])
flake28=np.array([20,4,3])
flake29=np.array([20,4,1])

In [ ]:
for file_name, number in sorted_files_with_numbers:
    file_path = os.path.join(folder_path, file_name) 
    globals()['fig_'+str(number)]=go.Figure()
    globals()['heatmap_fig_'+str(number)]=go.Figure()
    if number>21:   
        data = np.loadtxt(file_path)
        x=data[1:,0]; y=data[1:,1]
        wavelength = data[0,2:]
        intensity = data[1:,2:]
    
        # ranged_indices = np.where((wavelength>=570) & (wavelength<=650))[0]
        # wavelength = wavelength[ranged_indices]
        # intensity = intensity[:,ranged_indices]
        
        # print(data.shape,x.shape,y.shape,wavelength.shape,intensity.shape,intensity)
        
        max_intensity = np.max(intensity,axis=1)
        closest_index_intensity = np.argmin(np.abs(intensity - max_intensity.reshape(-1,1)),axis=1)
        lambda_matrix = np.tile(wavelength, (closest_index_intensity.shape[0],1))
        corr_lambda = lambda_matrix[np.arange(lambda_matrix.shape[0]), closest_index_intensity]
        zpl_indices = np.where(np.abs(corr_lambda - 580) <=6)[0]
        zpl_lambda = corr_lambda[zpl_indices]
        zpl_x = x[zpl_indices]
        zpl_y = y[zpl_indices]
        # print(closest_index_intensity.shape[0],x.shape)
        # print(zpl_lambda,zpl_x,zpl_y)
    
        #for heatmap plot 
        select_lambda = 586
        closest_index = np.argmin(np.abs(wavelength - select_lambda))
        spec_lambda = wavelength[closest_index]
        spec_intensity = intensity[:,closest_index]
        globals()['heatmap_fig_'+str(number)].add_scatter(x=x-np.mean(x),y=y-np.mean(y),
                                                  mode='markers',          
                                                    marker=dict(
                                                        size=13,       
                                                        color=spec_intensity,      
                                                        colorscale='Viridis', 
                                                        colorbar=dict(title=f'Intensity <br>(&#955; &#8764; {select_lambda}nm)</br>'),  
                                                        showscale=True       
                                                    ))
        flake_number = globals()['flake'+str(number)]
        globals()['heatmap_fig_'+str(number)].update_layout(template=fig_template,width = 800,height = 600,
                            title="Flake "+str(number)+f"<br>[{flake_number[0]}KeV, {flake_number[1]}&mu;m, {flake_number[2]}min]</br>",
                            xaxis_title=f"x ( + mean : {np.round(np.mean(x),2)}) &mu;m",
                            yaxis_title=f"y ( + mean : {np.round(np.mean(y),2)}) &mu;m",
                            legend_title="(x,y) coordinates",
                            font=dict(
                                family="Times New Roman, monospace",
                                size=18,
                                color="black")
                     )
        # globals()['heatmap_fig_'+str(number)].show()
    
        for index, (x_point, y_point) in enumerate(zip(x, y)):    
            if np.isin(x_point,zpl_x) and np.isin(y_point,zpl_y):
                globals()['fig_'+str(number)].add_scatter(x=wavelength,y=intensity[index,:],name = f'({np.round(x_point,2)},{np.round(y_point,2)})')
        globals()['fig_'+str(number)].update_layout(template=fig_template,width = 800,height = 600,
                            title="Flake "+str(number),
                            xaxis_title="wavelength (nm)",
                            yaxis_title="Intensity (arb. unit)",
                            legend_title="(x,y) coordinates",
                            font=dict(
                                family="Times New Roman, monospace",
                                size=18,
                                color="black")
                     )
        globals()['fig_'+str(number)].show()
        
        # pyo.plot(globals()['heatmap_fig_'+str(number)], filename=path + "/images/heatmap_flake_"+str(number)+".html", auto_open=False) 
        # pyo.plot(globals()['fig_'+str(number)], filename=path + "/images/flake_ZPL_"+str(number)+".html", auto_open=False) 